# AI Agents

In [ ]:
import os
from dotenv import load_dotenv
from langchain_openai import AzureOpenAI
from langchain_core.messages import HumanMessage
from langchain_openai import AzureChatOpenAI
import requests
from langchain.agents import AgentExecutor, create_openai_tools_agent
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain.tools import BaseTool, StructuredTool, tool

# Load environment variables from .env file
load_dotenv()

# Access the environment variables
openai_api_version = os.getenv("AZURE_OPENAI_API_VERSION")
azure_endpoint = os.getenv("AZURE_OPENAI_ENDPOINT")
openai_api_key = os.getenv("AZURE_OPENAI_API_KEY")
azure_chat_deployment = os.getenv("AZURE_OPENAI_CHAT_DEPLOYMENT_NAME")

# Initialize the Azure OpenAI model
model = AzureChatOpenAI(
    openai_api_version=openai_api_version,
    azure_deployment=azure_chat_deployment,
)

# Create a human message and invoke the model
message = HumanMessage(
    content="Translate this sentence from English to French. I love programming."
)
response = model.invoke([message])

# Print the response
print(response)


content="J'adore la programmation." additional_kwargs={'refusal': None} response_metadata={'token_usage': {'completion_tokens': 5, 'prompt_tokens': 19, 'total_tokens': 24, 'completion_tokens_details': None, 'prompt_tokens_details': None}, 'model_name': 'gpt-4o-2024-05-13', 'system_fingerprint': 'fp_04751d0b65', 'prompt_filter_results': [{'prompt_index': 0, 'content_filter_results': {'hate': {'filtered': False, 'severity': 'safe'}, 'jailbreak': {'filtered': False, 'detected': False}, 'self_harm': {'filtered': False, 'severity': 'safe'}, 'sexual': {'filtered': False, 'severity': 'safe'}, 'violence': {'filtered': False, 'severity': 'safe'}}}], 'finish_reason': 'stop', 'logprobs': None, 'content_filter_results': {'hate': {'filtered': False, 'severity': 'safe'}, 'protected_material_code': {'filtered': False, 'detected': False}, 'protected_material_text': {'filtered': False, 'detected': False}, 'self_harm': {'filtered': False, 'severity': 'safe'}, 'sexual': {'filtered': False, 'severity': 's

## SQL Agent

In [4]:
import sqlite3
import pandas as pd

# Sample data for the climbing product catalog
data = {
    "Product ID": ["P001", "P002", "P003", "P004", "P005"],
    "Product Name": ["Climbing Rope", "Climbing Shoes", "Carabiner", "Harness", "Chalk Bag"],
    "Category": ["Ropes", "Shoes", "Hardware", "Harnesses", "Accessories"],
    "Price": [100, 150, 20, 75, 15],
    "Stock": [50, 30, 100, 40, 200],
    "Description": [
        "Durable and strong climbing rope suitable for all terrains.",
        "High-performance climbing shoes for advanced climbers.",
        "Lightweight and secure carabiner for all climbing needs.",
        "Comfortable and adjustable harness for safety.",
        "Compact and durable chalk bag for better grip."
    ]
}

# Converting the dictionary to a DataFrame
df = pd.DataFrame(data)

# Connecting to a SQLite database (it will create a new one if it doesn't exist)
conn = sqlite3.connect('climbing_product_catalog.db')

# Storing the DataFrame in the SQLite database
df.to_sql('product_catalog', conn, index=False, if_exists='replace')

# Committing and closing the connection
conn.commit()
conn.close()

In [11]:
from langchain_community.utilities.sql_database import SQLDatabase
db = SQLDatabase.from_uri("sqlite:///climbing_product_catalog.db")

from langchain_community.agent_toolkits.sql.toolkit import SQLDatabaseToolkit

sql_toolkit = SQLDatabaseToolkit(db=db, llm=model)
sql_toolkit.get_tools()

[QuerySQLDataBaseTool(description="Input to this tool is a detailed and correct SQL query, output is a result from the database. If the query is not correct, an error message will be returned. If an error is returned, rewrite the query, check the query, and try again. If you encounter an issue with Unknown column 'xxxx' in 'field list', use sql_db_schema to query the correct table fields.", db=<langchain_community.utilities.sql_database.SQLDatabase object at 0x00000275CA440A70>),
 InfoSQLDatabaseTool(description='Input to this tool is a comma-separated list of tables, output is the schema and sample rows for those tables. Be sure that the tables actually exist by calling sql_db_list_tables first! Example Input: table1, table2, table3', db=<langchain_community.utilities.sql_database.SQLDatabase object at 0x00000275CA440A70>),
 ListSQLDatabaseTool(db=<langchain_community.utilities.sql_database.SQLDatabase object at 0x00000275CA440A70>),
 QuerySQLCheckerTool(description='Use this tool to 

## Add to cart tool

In [ ]:
# Define the tool to add an item to the cart
@tool
def add_to_cart(item_name: str, item_price: float) -> str:
    """Add an item to the cart."""
    url = 'http://localhost:3000/cart'  # Ensure this matches the JSON Server endpoint
    cart_item = {
        'name': item_name,
        'price': item_price
    }
    
    response = requests.post(url, json=cart_item)
    
    if response.status_code == 201:
        return f"Item '{item_name}' added to cart successfully."
    else:
        return f"Failed to add item to cart: {response.status_code} {response.text}"


## Prompt Design

In [12]:
# Define the prompt template
prompt = ChatPromptTemplate.from_messages(
    [
        ("system", """You are an assistant that can manage a shopping cart.
        You assist the user in deciding the best climbing item with iterative questions.
        When the user asks you for products information, you can query the database.
        Once the user is ready to purchase, ask whether they want to add the item to the cart. 
        If they agree, add the item to the cart leveraging your tools.
         
        When you use a tool, Return only the answers. e.g
        Human: Add Climbing Rope to the cart with price 100
        AI: Item 'Climbing Rope' added to cart successfully.
        """),
        MessagesPlaceholder("chat_history", optional=True),
        ("human", "{input}"),
        MessagesPlaceholder("agent_scratchpad"),
    ]
)

## Running the Agent

In [13]:
# Setup the toolkit
toolkit = [add_to_cart, sql_toolkit.get_tools()[0], sql_toolkit.get_tools()[1], sql_toolkit.get_tools()[2], sql_toolkit.get_tools()[3]]

from langchain_community.chat_message_histories import ChatMessageHistory
from langchain_core.runnables.history import RunnableWithMessageHistory
 
message_history = ChatMessageHistory()

# Construct the OpenAI Tools agent
agent = create_openai_tools_agent(model, toolkit, prompt)

# Create an agent executor by passing in the agent and tools
agent_executor = AgentExecutor(agent=agent, tools=toolkit, verbose=True)

agent_with_chat_history = RunnableWithMessageHistory(
    agent_executor,
    # This is needed because in most real world scenarios, a session id is needed
    # It isn't really used here because we are using a simple in memory ChatMessageHistory
    lambda session_id: message_history,
    input_messages_key="input",
    history_messages_key="chat_history",
)
 


# Interactive loop
while True:
    user_input = input("Type your question here (or type 'exit' to quit): ")
    if user_input.lower() == 'exit':
        break
    result = agent_with_chat_history.invoke(
        {"input": user_input},
        # This is needed because in most real world scenarios, a session id is needed
        # It isn't really used here because we are using a simple in memory ChatMessageHistory
        config={"configurable": {"session_id": "<foo>"}},
    )
    print(f'User: {user_input}')
    print(f'AI: {result['output']}')



> Entering new AgentExecutor chain...
Hello! How can I assist you today? Are you looking for some climbing gear or need help with something else?

> Finished chain.
User: hi
AI: Hello! How can I assist you today? Are you looking for some climbing gear or need help with something else?


> Entering new AgentExecutor chain...

Invoking: `sql_db_list_tables` with `{}`
responded: Let me check the available products for you.

product_catalog
Invoking: `sql_db_schema` with `{'table_names': 'product_catalog'}`



CREATE TABLE product_catalog (
	"Product ID" TEXT, 
	"Product Name" TEXT, 
	"Category" TEXT, 
	"Price" INTEGER, 
	"Stock" INTEGER, 
	"Description" TEXT
)

/*
3 rows from product_catalog table:
Product ID	Product Name	Category	Price	Stock	Description
P001	Climbing Rope	Ropes	100	50	Durable and strong climbing rope suitable for all terrains.
P002	Climbing Shoes	Shoes	150	30	High-performance climbing shoes for advanced climbers.
P003	Carabiner	Hardware	20	100	Lightweight and secure ca